<a href="https://colab.research.google.com/github/esthermead/deeplearning/blob/master/migrants_crowd_counting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# my attempt to use my own image dataset.
# original tutorial from https://www.analyticsvidhya.com/blog/2019/02/building-crowd-counting-model-python/
# have to change to python 2 to work (in change runtime tab at top of colab window).
# when change from p3 to p2 you will have to change working directory again.

In [1]:
#every time make new colab, do these:
# click Runtime, select "change runtime", choose "GPU" accelaration.
# mount drive
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [2]:
#and change working directory what what you need:
# change working directory to my DeepLearning folder and current project
cd "/content/drive/My Drive/DeepLearning/CSRNet-pytorch"

/content/drive/My Drive/DeepLearning/CSRNet-pytorch


In [0]:
#now just copying what worked from the crowd counting tut (url above).
# importing libraries
import h5py
import scipy.io as io
import PIL.Image as Image
import numpy as np
import os
import glob
from matplotlib import pyplot as plt
from scipy.ndimage.filters import gaussian_filter
import scipy
import json
from matplotlib import cm as CM
from image import *
from model import CSRNet
import torch
from tqdm import tqdm
%matplotlib inline

In [0]:
# function to create density maps for images
# code on tut is wrong; copy from "make_dataset.ipynb" instead
#this is borrowed from https://github.com/davideverona/deep-crowd-counting_crowdnet
def gaussian_filter_density(gt):
    print gt.shape
    density = np.zeros(gt.shape, dtype=np.float32)
    gt_count = np.count_nonzero(gt)
    if gt_count == 0:
        return density

    pts = np.array(zip(np.nonzero(gt)[1], np.nonzero(gt)[0]))
    leafsize = 2048
    # build kdtree
    tree = scipy.spatial.KDTree(pts.copy(), leafsize=leafsize)
    # query kdtree
    distances, locations = tree.query(pts, k=4)

    print 'generate density...'
    for i, pt in enumerate(pts):
        pt2d = np.zeros(gt.shape, dtype=np.float32)
        pt2d[pt[1],pt[0]] = 1.
        if gt_count > 1:
            sigma = (distances[i][1]+distances[i][2]+distances[i][3])*0.1
        else:
            sigma = np.average(np.array(gt.shape))/2./2. #case: 1 point
        density += scipy.ndimage.filters.gaussian_filter(pt2d, sigma, mode='constant')
    print 'done.'
    return density

In [0]:
#set the root to the migrant dataset you download
root = '/content/drive/My Drive/DeepLearning/CSRNet-pytorch/'

In [0]:
#THIS IS WHERE I HAVE TO FIGURE OUT
#i don't understand why they made a part A and part B. but i did it.
#now generate the migrants' ground truth?
#had to change these paths
part_A_train = os.path.join(root,'migrants/part_A/train_data','images')
part_A_test = os.path.join(root,'migrants/part_A/test_data','images')
part_B_train = os.path.join(root,'migrants/part_B/train_data','images')
part_B_test = os.path.join(root,'migrants/part_B/test_data','images')
path_sets = [part_A_train,part_A_test]

In [0]:
img_paths = []
for path in path_sets:
    for img_path in glob.glob(os.path.join(path, '*.jpg')):
        img_paths.append(img_path)

In [8]:
for img_path in img_paths:
    print (img_path)
    mat = io.loadmat(img_path.replace('.jpg','.mat').replace('images','ground-truth').replace('IMG_','GT_IMG_'))
    img= plt.imread(img_path)
    k = np.zeros((img.shape[0],img.shape[1]))
    gt = mat["image_info"][0,0][0,0][0]
    for i in range(0,len(gt)):
        if int(gt[i][1])<img.shape[0] and int(gt[i][0])<img.shape[1]:
            k[int(gt[i][1]),int(gt[i][0])]=1
    k = gaussian_filter_density(k)
    with h5py.File(img_path.replace('.jpg','.h5').replace('images','ground-truth'), 'w') as hf:
            hf['density'] = k

/content/drive/My Drive/DeepLearning/CSRNet-pytorch/migrants/part_A/train_data/images/00000000.jpg


IOError: ignored

In [0]:
#had to restart chrome bcz couldn't get into BB classroom.
#pick back up here when return: https://www.analyticsvidhya.com/blog/2019/02/building-crowd-counting-model-python/